In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GroupKFold
from lightgbm import LGBMClassifier
import os

## Definition of path and downloading the data
As sometimes I run the notebook on kaggle and sometimes on my PC I must change the path.

In [2]:
#global variables
KAGGLE = 1
PATH = "/kaggle/input/" if KAGGLE ==1 else "C:/Users/pzlpa_000/"

In [3]:
df_train = pd.read_csv( os.path.join(PATH,"isic-2024-challenge/train-metadata.csv"))
df_test = pd.read_csv( os.path.join(PATH,"isic-2024-challenge/test-metadata.csv"))

/tmp/ipykernel_17/3060065471.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv( os.path.join(PATH,"isic-2024-challenge/train-metadata.csv"))


In [4]:
df_test=df_train[200000:]
df_train=df_train[:200000]

In [5]:
df_train

,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,lesion_id,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence
0,ISIC_0015670,0,IP_1235828,60.0,male,lower extremity,3.04,TBP tile: close-up,3D: white,20.244422,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,97.517282
1,ISIC_0015845,0,IP_8170065,60.0,male,head/neck,1.10,TBP tile: close-up,3D: white,31.712570,...,IL_6727506,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,3.141455
2,ISIC_0015864,0,IP_6724798,60.0,male,posterior torso,3.40,TBP tile: close-up,3D: XP,22.575830,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.804040
3,ISIC_0015902,0,IP_4111386,65.0,male,anterior torso,3.22,TBP tile: close-up,3D: XP,14.242329,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.989998
4,ISIC_0024200,0,IP_8313778,55.0,male,anterior torso,2.73,TBP tile: close-up,3D: white,24.725520,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,70.442510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,ISIC_5031486,0,IP_4860795,65.0,male,posterior torso,3.30,TBP tile: close-up,3D: XP,15.757896,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.906343
199996,ISIC_5031489,0,IP_0669361,70.0,male,posterior torso,2.57,TBP tile: close-up,3D: XP,26.499412,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.999583
199997,ISIC_5031493,0,IP_1127121,65.0,male,NaN,2.96,TBP tile: close-up,3D: XP,22.260339,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,94.238710
199998,ISIC_5031587,0,IP_7655469,30.0,male,anterior torso,4.29,TBP tile: close-up,3D: white,21.125730,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,100.000000


In [6]:
df_test

,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,lesion_id,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence
200000,ISIC_5031612,0,IP_4106376,65.0,male,posterior torso,3.43,TBP tile: close-up,3D: XP,23.998817,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.994302
200001,ISIC_5031634,0,IP_0669361,70.0,male,anterior torso,2.88,TBP tile: close-up,3D: XP,12.829299,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.999094
200002,ISIC_5031649,0,IP_0218255,65.0,male,upper extremity,2.81,TBP tile: close-up,3D: XP,17.580419,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.999416
200003,ISIC_5031662,0,IP_2529526,45.0,male,lower extremity,3.59,TBP tile: close-up,3D: XP,22.006207,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.935383
200004,ISIC_5031681,0,IP_6796597,50.0,female,lower extremity,3.87,TBP tile: close-up,3D: XP,18.071040,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401054,ISIC_9999937,0,IP_1140263,70.0,male,anterior torso,6.80,TBP tile: close-up,3D: XP,22.574335,...,IL_9520694,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.999988
401055,ISIC_9999951,0,IP_5678181,60.0,male,posterior torso,3.11,TBP tile: close-up,3D: white,19.977640,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.999820
401056,ISIC_9999960,0,IP_0076153,65.0,female,anterior torso,2.05,TBP tile: close-up,3D: XP,17.332567,...,IL_9852274,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.999416
401057,ISIC_9999964,0,IP_5231513,30.0,female,anterior torso,2.80,TBP tile: close-up,3D: XP,22.288570,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,100.000000


## Explonatory DF
I create a df with columns description to consult it if necessary inside the notebook.

In [7]:
data = {
    "field_name": [
        "target", "lesion_id", "iddx_full", "iddx_1", "iddx_2", "iddx_3", "iddx_4", "iddx_5", 
        "mel_mitotic_index", "mel_thick_mm", "tbp_lv_dnn_lesion_confidence", 
        "isic_id", "patient_id", "age_approx", "sex", "anatom_site_general", 
        "clin_size_long_diam_mm", "image_type", "tbp_tile_type", "tbp_lv_A", "tbp_lv_Aext", 
        "tbp_lv_B", "tbp_lv_Bext", "tbp_lv_C", "tbp_lv_Cext", "tbp_lv_H", "tbp_lv_Hext", 
        "tbp_lv_L", "tbp_lv_Lext", "tbp_lv_areaMM2", "tbp_lv_area_perim_ratio", 
        "tbp_lv_color_std_mean", "tbp_lv_deltaA", "tbp_lv_deltaB", "tbp_lv_deltaL", 
        "tbp_lv_deltaLBnorm", "tbp_lv_eccentricity", "tbp_lv_location", 
        "tbp_lv_location_simple", "tbp_lv_minorAxisMM", "tbp_lv_nevi_confidence", 
        "tbp_lv_norm_border", "tbp_lv_norm_color", "tbp_lv_perimeterMM", 
        "tbp_lv_radial_color_std_max", "tbp_lv_stdL", "tbp_lv_stdLExt", 
        "tbp_lv_symm_2axis", "tbp_lv_symm_2axis_angle", "tbp_lv_x", "tbp_lv_y", "tbp_lv_z", 
        "attribution", "copyright_license"
    ],
    "description": [
        "Binary class {0: benign, 1: malignant}.", 
        "Unique lesion identifier. Present in lesions that were manually tagged as a lesion of interest.", 
        "Fully classified lesion diagnosis.", 
        "First level lesion diagnosis.", 
        "Second level lesion diagnosis.", 
        "Third level lesion diagnosis.", 
        "Fourth level lesion diagnosis.", 
        "Fifth level lesion diagnosis.", 
        "Mitotic index of invasive malignant melanomas.", 
        "Thickness in depth of melanoma invasion.", 
        "Lesion confidence score (0-100 scale).", 
        "Unique case identifier.", 
        "Unique patient identifier.", 
        "Approximate age of patient at time of imaging.", 
        "Sex of the person.", 
        "Location of the lesion on the patient's body.", 
        "Maximum diameter of the lesion (mm).", 
        "Structured field of the ISIC Archive for image type.", 
        "Lighting modality of the 3D TBP source image.", 
        "A inside lesion.", 
        "A outside lesion.", 
        "B inside lesion.", 
        "B outside lesion.", 
        "Chroma inside lesion.", 
        "Chroma outside lesion.", 
        "Hue inside the lesion; calculated as the angle of A* and B* in LAB* color space. Typical values range from 25 (red) to 75 (brown).", 
        "Hue outside lesion.", 
        "L inside lesion.", 
        "L outside lesion.", 
        "Area of lesion (mm^2).", 
        "Border jaggedness, the ratio between lesions perimeter and area. Circular lesions will have low values; irregular shaped lesions will have higher values. Values range 0-10.", 
        "Color irregularity, calculated as the variance of colors within the lesion's boundary.", 
        "Average A contrast (inside vs. outside lesion).", 
        "Average B contrast (inside vs. outside lesion).", 
        "Average L contrast (inside vs. outside lesion).", 
        "Contrast between the lesion and its immediate surrounding skin. Low contrast lesions tend to be faintly visible such as freckles; high contrast lesions tend to be those with darker pigment. Calculated as the average delta LB of the lesion relative to its immediate background in LAB* color space. Typical values range from 5.5 to 25.", 
        "Eccentricity.", 
        "Classification of anatomical location, divides arms & legs to upper & lower; torso into thirds.", 
        "Classification of anatomical location, simple.", 
        "Smallest lesion diameter (mm).", 
        "Nevus confidence score (0-100 scale) is a convolutional neural network classifier estimated probability that the lesion is a nevus. The neural network was trained on approximately 57,000 lesions that were classified and labeled by a dermatologist.", 
        "Border irregularity (0-10 scale); the normalized average of border jaggedness and asymmetry.", 
        "Color variation (0-10 scale); the normalized average of color asymmetry and color irregularity.", 
        "Perimeter of lesion (mm).", 
        "Color asymmetry, a measure of asymmetry of the spatial distribution of color within the lesion. This score is calculated by looking at the average standard deviation in LAB* color space within concentric rings originating from the lesion center. Values range 0-10.", 
        "Standard deviation of L inside lesion.", 
        "Standard deviation of L outside lesion.", 
        "Border asymmetry; a measure of asymmetry of the lesion's contour about an axis perpendicular to the lesion's most symmetric axis. Lesions with two axes of symmetry will therefore have low scores (more symmetric), while lesions with only one or zero axes of symmetry will have higher scores (less symmetric). This score is calculated by comparing opposite halves of the lesion contour over many degrees of rotation. The angle where the halves are most similar identifies the principal axis of symmetry, while the second axis of symmetry is perpendicular to the principal axis. Border asymmetry is reported as the asymmetry value about this second axis. Values range 0-10.", 
        "Lesion border asymmetry angle.", 
        "X-coordinate of the lesion on 3D TBP.", 
        "Y-coordinate of the lesion on 3D TBP.", 
        "Z-coordinate of the lesion on 3D TBP.", 
        "Image attribution, synonymous with image source.", 
        "Copyright license."
    ],
    "train_only": [
        True, True, True, True, True, True, True, True, True, True, True,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False
    ]
}

df_explained = pd.DataFrame(data)


In [8]:
pd.set_option('display.max_colwidth', None)
print(df_explained['description'][df_explained['field_name']=="tbp_lv_color_std_mean"])

31    Color irregularity, calculated as the variance of colors within the lesion's boundary.
Name: description, dtype: object


In [9]:
#creating a variable with columns that appear both in test and train
test_and_target=["target", 
        "isic_id", "patient_id", "age_approx", "sex", "anatom_site_general", 
        "clin_size_long_diam_mm", "image_type", "tbp_tile_type", "tbp_lv_A", "tbp_lv_Aext", 
        "tbp_lv_B", "tbp_lv_Bext", "tbp_lv_C", "tbp_lv_Cext", "tbp_lv_H", "tbp_lv_Hext", 
        "tbp_lv_L", "tbp_lv_Lext", "tbp_lv_areaMM2", "tbp_lv_area_perim_ratio", 
        "tbp_lv_color_std_mean", "tbp_lv_deltaA", "tbp_lv_deltaB", "tbp_lv_deltaL", 
        "tbp_lv_deltaLBnorm", "tbp_lv_eccentricity", "tbp_lv_location", 
        "tbp_lv_location_simple", "tbp_lv_minorAxisMM", "tbp_lv_nevi_confidence", 
        "tbp_lv_norm_border", "tbp_lv_norm_color", "tbp_lv_perimeterMM", 
        "tbp_lv_radial_color_std_max", "tbp_lv_stdL", "tbp_lv_stdLExt", 
        "tbp_lv_symm_2axis", "tbp_lv_symm_2axis_angle", "tbp_lv_x", "tbp_lv_y", "tbp_lv_z", 
        "attribution", "copyright_license"
    ]

In [10]:
#creating a df with columns that appear both in train and test
df_train_filtered=df_train[test_and_target]
categorical_columns = df_train_filtered.select_dtypes(include=['object', 'category']).columns
numerical_columns = df_train_filtered.select_dtypes(include=['number']).columns

## EDA


In [11]:
# checking columns with zero values as they can be really null values
def value_counts (df, value):
    value_columns = df.columns[(df == value).any()]
    print(value_columns)

    # Count number of 0s in each column that has at least one 0
    value_counts = df[value_columns].eq(0).sum()

    # Print the number of 0s in each relevant column
    print(f"Number of {value} in each column that has at least one {value}:{value_counts}")

value_counts(df_train_filtered,0)
 

Index(['target', 'tbp_lv_color_std_mean', 'tbp_lv_nevi_confidence',
       'tbp_lv_norm_color', 'tbp_lv_radial_color_std_max',
       'tbp_lv_symm_2axis_angle'],
      dtype='object')
Number of 0 in each column that has at least one 0:target                         199810
tbp_lv_color_std_mean           13031
tbp_lv_nevi_confidence              4
tbp_lv_norm_color               13028
tbp_lv_radial_color_std_max     14815
tbp_lv_symm_2axis_angle          9271
dtype: int64


In [12]:
# Calculate mean for numerical columns grouped by 'target'
means_by_target = df_train_filtered[numerical_columns].groupby(df_train_filtered['target']).mean()
print(means_by_target)

# Check the number of different values in cat columns to see which cat columns can be useful
for c in categorical_columns:
    num_dif = df_train_filtered[c].nunique()
    if num_dif < 30:
        print (f'{c}: {num_dif} different values : {df_train_filtered[c].unique()}')
    else:
        print (f'{c}: {num_dif} different values')             
    

        target  age_approx  clin_size_long_diam_mm   tbp_lv_A  tbp_lv_Aext  \
target                                                                       
0          0.0   58.024462                3.927329  19.970833    14.920293   
1          1.0   61.436170                5.711895  21.994067    17.102991   

         tbp_lv_B  tbp_lv_Bext   tbp_lv_C  tbp_lv_Cext   tbp_lv_H  ...  \
target                                                             ...   
0       28.281381    26.909527  34.783586    30.918415  54.659963  ...   
1       23.619414    24.834587  32.598185    30.460477  47.040209  ...   

        tbp_lv_norm_color  tbp_lv_perimeterMM  tbp_lv_radial_color_std_max  \
target                                                                       
0                3.092027           11.863051                     1.017284   
1                4.345789           18.809479                     1.526252   

        tbp_lv_stdL  tbp_lv_stdLExt  tbp_lv_symm_2axis  \
target             

In [13]:
# Check distribution of one cat variable for target 0 and 1
place_by_target = df_train_filtered.groupby(['anatom_site_general', 'target']).size().unstack(fill_value=0)
place_totals = place_by_target.sum(axis=1)
place_by_target['percentage_1'] = (place_by_target[1] / place_totals) * 100
print(place_by_target)

target                   0   1  percentage_1
anatom_site_general                         
anterior torso       43747  38      0.086788
head/neck             6097  37      0.603195
lower extremity      51144  34      0.066435
posterior torso      60773  52      0.085491
upper extremity      35203  29      0.082312


## Preprocessing

In [14]:
df_train_filtered = df_train_filtered[df_train_filtered['anatom_site_general'].notna()]
df_train_filtered.reset_index(drop=True, inplace=True)

In [15]:
def feature_engineering(df):
    # Taken from https://www.kaggle.com/code/snnclsr/tabular-ensemble-lgbm-catboost
    df["lesion_size_ratio"] = df["tbp_lv_minorAxisMM"] / df["clin_size_long_diam_mm"]
    df["lesion_shape_index"] = df["tbp_lv_areaMM2"] / (df["tbp_lv_perimeterMM"] ** 2)
    df["hue_contrast"] = (df["tbp_lv_H"] - df["tbp_lv_Hext"]).abs()
    df["luminance_contrast"] = (df["tbp_lv_L"] - df["tbp_lv_Lext"]).abs()
    df["lesion_color_difference"] = np.sqrt(df["tbp_lv_deltaA"] ** 2 + df["tbp_lv_deltaB"] ** 2 + df["tbp_lv_deltaL"] ** 2)
    df["border_complexity"] = df["tbp_lv_norm_border"] + df["tbp_lv_symm_2axis"]
    df["color_uniformity"] = df["tbp_lv_color_std_mean"] / df["tbp_lv_radial_color_std_max"]
    df["3d_position_distance"] = np.sqrt(df["tbp_lv_x"] ** 2 + df["tbp_lv_y"] ** 2 + df["tbp_lv_z"] ** 2) 
    df["perimeter_to_area_ratio"] = df["tbp_lv_perimeterMM"] / df["tbp_lv_areaMM2"]
    df["lesion_visibility_score"] = df["tbp_lv_deltaLBnorm"] + df["tbp_lv_norm_color"] 
    df["symmetry_border_consistency"] = df["tbp_lv_symm_2axis"] * df["tbp_lv_norm_border"]
    df["color_consistency"] = df["tbp_lv_stdL"] / df["tbp_lv_Lext"]    
    df["size_age_interaction"] = df["clin_size_long_diam_mm"] * df["age_approx"]
    df["hue_color_std_interaction"] = df["tbp_lv_H"] * df["tbp_lv_color_std_mean"]
    df["lesion_severity_index"] = (df["tbp_lv_norm_border"] + df["tbp_lv_norm_color"] + df["tbp_lv_eccentricity"]) / 3
    df["shape_complexity_index"] = df["border_complexity"] + df["lesion_shape_index"]
    df["color_contrast_index"] = df["tbp_lv_deltaA"] + df["tbp_lv_deltaB"] + df["tbp_lv_deltaL"] + df["tbp_lv_deltaLBnorm"]
    df["log_lesion_area"] = np.log(df["tbp_lv_areaMM2"] + 1)
    df["normalized_lesion_size"] = df["clin_size_long_diam_mm"] / df["age_approx"]
    df["mean_hue_difference"] = (df["tbp_lv_H"] + df["tbp_lv_Hext"]) / 2
    df["std_dev_contrast"] = np.sqrt((df["tbp_lv_deltaA"] ** 2 + df["tbp_lv_deltaB"] ** 2 + df["tbp_lv_deltaL"] ** 2) / 3)
    df["color_shape_composite_index"] = (df["tbp_lv_color_std_mean"] + df["tbp_lv_area_perim_ratio"] + df["tbp_lv_symm_2axis"]) / 3
    df["3d_lesion_orientation"] = np.arctan2(df_train["tbp_lv_y"], df_train["tbp_lv_x"])
    df["overall_color_difference"] = (df["tbp_lv_deltaA"] + df["tbp_lv_deltaB"] + df["tbp_lv_deltaL"]) / 3
    df["symmetry_perimeter_interaction"] = df["tbp_lv_symm_2axis"] * df["tbp_lv_perimeterMM"]
    df["comprehensive_lesion_index"] = (df["tbp_lv_area_perim_ratio"] + df["tbp_lv_eccentricity"] + df["tbp_lv_norm_color"] + df["tbp_lv_symm_2axis"]) / 4
    df["color_variance_ratio"] = df["tbp_lv_color_std_mean"] / df["tbp_lv_stdLExt"]
    df["border_color_interaction"] = df["tbp_lv_norm_border"] * df["tbp_lv_norm_color"]
    df["size_color_contrast_ratio"] = df["clin_size_long_diam_mm"] / df["tbp_lv_deltaLBnorm"]
    df["age_normalized_nevi_confidence"] = df["tbp_lv_nevi_confidence"] / df["age_approx"]
    df["color_asymmetry_index"] = df["tbp_lv_radial_color_std_max"] * df["tbp_lv_symm_2axis"]
    df["3d_volume_approximation"] = df["tbp_lv_areaMM2"] * np.sqrt(df["tbp_lv_x"]**2 + df["tbp_lv_y"]**2 + df["tbp_lv_z"]**2)
    df["color_range"] = (df["tbp_lv_L"] - df["tbp_lv_Lext"]).abs() + (df["tbp_lv_A"] - df["tbp_lv_Aext"]).abs() + (df["tbp_lv_B"] - df["tbp_lv_Bext"]).abs()
    df["shape_color_consistency"] = df["tbp_lv_eccentricity"] * df["tbp_lv_color_std_mean"]
    df["border_length_ratio"] = df["tbp_lv_perimeterMM"] / (2 * np.pi * np.sqrt(df["tbp_lv_areaMM2"] / np.pi))
    df["age_size_symmetry_index"] = df["age_approx"] * df["clin_size_long_diam_mm"] * df["tbp_lv_symm_2axis"]   
    return df

In [16]:
def feature_engineering_process(df, num_columns, medians):
     # Handle numerical columns missing values
    for nc in num_columns:
        df[nc] = df[nc].fillna(medians.get(nc, 0))  
    # Replacement for 0 to na to avoid incorrect calulation in feature engineering as 0 really mean missing value
    replace_cols = ['tbp_lv_color_std_mean', 'tbp_lv_norm_color', 'tbp_lv_radial_color_std_max']
    for col in replace_cols:
        if col in df.columns:
            df[col] = df[col].replace(0, np.nan)
    #Feature engineering
    df_num = df[num_columns]
    df_num = feature_engineering(df_num)   
    numerical_columns_list = list(df_num.columns)    
    
    return df_num, numerical_columns_list

In [17]:
class Preprocessor:
    def __init__(self):
        self.medians = {}
        self.one_hot_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
        self.scaler = StandardScaler()
        self.encoded_cols = []
        self.numerical_columns = []
        self.cat_cols = ["sex", "anatom_site_general"]       

    def fit(self, df):
        # Check if input is a numpy array and convert to DataFrame if necessary
        if isinstance(df, np.ndarray):
            df = pd.DataFrame(df)
        
        # Determine numerical columns
        self.numerical_columns = list(df.select_dtypes(include=['number']).columns)       
        if "target" in self.numerical_columns:
            self.numerical_columns.remove("target") 
        
        # Compute medians for numerical columns
        num_medians = df[self.numerical_columns].median()
        self.medians = dict(num_medians)

        # Compute median for angle
        if 'tbp_lv_symm_2axis_angle' in df.columns:
            self.medians['tbp_lv_symm_2axis_angle'] = df['tbp_lv_symm_2axis_angle'].median()
        
        # Fit OneHotEncoder
        self.one_hot_encoder.fit(df[self.cat_cols])
        self.encoded_cols = self.one_hot_encoder.get_feature_names_out(self.cat_cols)                 
        
        # I have to do feature engineering process in fit as I nedd the result to fit the scaler
        #Feature engineering
        df, numerical_columns_list = feature_engineering_process (df,self.numerical_columns,self.medians)      
        
        # Scale numerical columns           
        self.scaler.fit(df[numerical_columns_list])

        return self

    def transform(self, df):
        # Handle categorical columns missing values
        df['sex'] = df['sex'].fillna('male')
        df[self.cat_cols] = df[self.cat_cols].fillna('other')            
        
        # Specific replacement for 'tbp_lv_symm_2axis_angle' as 0 mean really missing value
        if 'tbp_lv_symm_2axis_angle' in df.columns:
            df['tbp_lv_symm_2axis_angle'] = df['tbp_lv_symm_2axis_angle'].replace(0, self.medians['tbp_lv_symm_2axis_angle'])        
       
        # Apply one-hot encoding
        X_cat = self.one_hot_encoder.transform(df[self.cat_cols])
        df_encoded = pd.DataFrame(X_cat, columns=self.encoded_cols)
        
        #Feature engineering
        df_num, numerical_columns_list = feature_engineering_process (df,self.numerical_columns,self.medians)          
        
        # Scale numerical columns       
        df_num[numerical_columns_list] = self.scaler.transform(df_num[numerical_columns_list])
        
        # Combine encoded and scaled data  
        df_final = pd.concat([df_num, df_encoded], axis=1)
        
        # Add id columns and target back if they exist
        if "isic_id" in df.columns and "patient_id" in df.columns:
            ids = df[["isic_id", "patient_id"]]
            df_final = pd.concat([df_final, ids], axis=1)
        if "target" in df.columns:
            targets = df[["target"]]
            df_final = pd.concat([df_final, targets], axis=1)
        
        return df_final
    

In [18]:
preprocessor = Preprocessor()
preprocessor.fit(df_train_filtered)
df_train_final = preprocessor.transform(df_train_filtered)

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/tmp/ipykernel_17/3122119603.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["lesion_size_ratio"] = df["tbp_lv_minorAxisMM"] / df["clin_size_long_diam_mm"]
/tmp/ipykernel_17/3122119603.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [19]:
len(df_train_final.columns)

80

## Training

### Competition metric

In [20]:
def comp_score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, min_tpr: float=0.80):
    v_gt = abs(np.asarray(solution.values)-1)
    v_pred = np.array([1.0 - x for x in submission.values])
    max_fpr = abs(1-min_tpr)
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
    # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    return partial_auc

### CV validation and shuffling

In [21]:
N_SPLITS = 10
gkf = GroupKFold(n_splits=N_SPLITS)

df_train_final = df_train_final.sample(frac=1).reset_index(drop=True)
df_train_final["fold"] = -1
for idx, (train_idx, val_idx) in enumerate(gkf.split(df_train_final, df_train_final["target"], groups=df_train_final["patient_id"])):
    df_train_final.loc[val_idx, "fold"] = idx
df_train_final

,age_approx,clin_size_long_diam_mm,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,tbp_lv_Cext,tbp_lv_H,tbp_lv_Hext,...,sex_nan,anatom_site_general_anterior torso,anatom_site_general_head/neck,anatom_site_general_lower extremity,anatom_site_general_posterior torso,anatom_site_general_upper extremity,isic_id,patient_id,target,fold
0,-1.325705,0.259587,-0.615090,-0.477415,-0.204583,-0.364580,-0.426760,-0.494867,0.466219,0.243774,...,0.0,0.0,0.0,1.0,0.0,0.0,ISIC_0360168,IP_5714646,0,1
1,0.517667,-0.148120,1.041574,0.918175,0.471419,0.160028,0.757999,0.444855,-0.504295,-0.767430,...,0.0,0.0,0.0,0.0,1.0,0.0,ISIC_1765758,IP_5426188,0,8
2,-0.219682,-0.418011,-0.389455,-0.748654,1.552909,1.461066,1.062652,0.975494,1.549478,1.570718,...,0.0,0.0,0.0,0.0,1.0,0.0,ISIC_2137945,IP_9472195,0,4
3,-0.588356,1.097970,1.657280,-0.341450,1.781336,1.934889,1.986891,1.492018,0.018673,1.405049,...,0.0,1.0,0.0,0.0,0.0,0.0,ISIC_0445542,IP_7424727,0,7
4,-0.219682,0.173451,0.667714,1.367346,0.282820,0.728954,0.459607,1.066738,-0.347659,-0.747557,...,0.0,0.0,0.0,0.0,1.0,0.0,ISIC_0592696,IP_7770500,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197149,-0.219682,-0.819976,-0.759964,-0.767834,0.688802,0.696395,0.234266,0.308961,1.339938,1.220812,...,0.0,0.0,0.0,0.0,0.0,1.0,ISIC_2431320,IP_0753755,0,5
197150,1.623690,4.945347,-0.749029,-0.894290,0.646133,0.649605,0.203110,0.234172,1.298315,1.327170,...,0.0,0.0,0.0,0.0,1.0,0.0,ISIC_2333104,IP_6474130,0,7
197151,-0.588356,0.690263,0.441284,0.440346,-0.572331,-0.584280,-0.254651,-0.323853,-0.972906,-0.911662,...,0.0,0.0,0.0,0.0,0.0,1.0,ISIC_1576252,IP_3260480,0,0
197152,0.517667,-0.521373,1.167948,0.689190,1.674527,1.727546,1.711202,1.622192,0.316127,0.423528,...,0.0,0.0,0.0,0.0,1.0,0.0,ISIC_4213246,IP_0528395,0,0


### training

In [22]:
lgb_params = {
    'objective': 'binary',
    # "random_state": 42,
    "n_estimators": 1500,
    'learning_rate': 0.001,
    'bagging_freq': 1,
    'pos_bagging_fraction': 0.75,
    'neg_bagging_fraction': 0.05,
    'feature_fraction': 0.6,
    'lambda_l1': 0.2,
    'lambda_l2': 0.7,
    'num_leaves': 35,
    "min_data_in_leaf": 50,
    "verbosity": -1,
    # "device": "gpu"
    # "extra_trees": True
}

lgb_scores = []
lgb_models = []
train_cols = list(df_train_final.columns)
for c in ["fold", "target","isic_id", "patient_id"]:
    train_cols.remove(c)
for fold in range(N_SPLITS):
    _df_train = df_train_final[df_train_final["fold"] != fold].reset_index(drop=True)
    _df_valid = df_train_final[df_train_final["fold"] == fold].reset_index(drop=True)
    model = LGBMClassifier(**lgb_params)    
    # model = VotingClassifier([(f"lgb_{i}", lgb.LGBMClassifier(random_state=i, **lgb_params)) for i in range(1)], voting="soft")
    model.fit(_df_train[train_cols], _df_train["target"])
    preds = model.predict_proba(_df_valid[train_cols])[:, 1]
    score = comp_score(_df_valid[["target"]], pd.DataFrame(preds, columns=["prediction"]), "")
    print(f"fold: {fold} - Partial AUC Score: {score:.5f}")
    lgb_scores.append(score)
    lgb_models.append(model)

fold: 0 - Partial AUC Score: 0.19277
fold: 1 - Partial AUC Score: 0.12066
fold: 2 - Partial AUC Score: 0.15549
fold: 3 - Partial AUC Score: 0.09410
fold: 4 - Partial AUC Score: 0.14456
fold: 5 - Partial AUC Score: 0.15991
fold: 6 - Partial AUC Score: 0.18062
fold: 7 - Partial AUC Score: 0.13491
fold: 8 - Partial AUC Score: 0.15512
fold: 9 - Partial AUC Score: 0.12454


In [23]:
lgbm_score = np.mean(lgb_scores)
print(f"LGBM Score: {lgbm_score:.5f}")

LGBM Score: 0.14627


### Prediction

In [24]:
df_test_final = preprocessor.transform(df_test)
test_cols = list(df_test_final.columns)
for c in ["isic_id", "patient_id", "target"]:
    if c in test_cols:
        test_cols.remove(c)
print(df_test_final[test_cols])
lgb_preds =np.mean([model.predict_proba(df_test_final[test_cols])[:, 1] for model in lgb_models],axis=0)
lgb_preds 

/tmp/ipykernel_17/3122119603.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["lesion_size_ratio"] = df["tbp_lv_minorAxisMM"] / df["clin_size_long_diam_mm"]
/tmp/ipykernel_17/3122119603.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["lesion_shape_index"] = df["tbp_lv_areaMM2"] / (df["tbp_lv_perimeterMM"] ** 2)
/tmp/ipykernel_17/3122119603.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

        age_approx  clin_size_long_diam_mm  tbp_lv_A  tbp_lv_Aext  tbp_lv_B  \
200000    0.517667               -0.285937  1.009992     0.952281  3.321944   
200001    0.886341               -0.601766 -1.784650    -1.661557 -0.998076   
200002    0.517667               -0.641963 -0.595908    -0.405295  0.080590   
200003   -0.957030               -0.194059  0.511436     0.672097  0.541215   
200004   -0.588356               -0.033273 -0.473153    -1.124621 -0.275979   
...            ...                     ...       ...          ...       ...   
199995         NaN                     NaN       NaN          NaN       NaN   
199996         NaN                     NaN       NaN          NaN       NaN   
199997         NaN                     NaN       NaN          NaN       NaN   
199998         NaN                     NaN       NaN          NaN       NaN   
199999         NaN                     NaN       NaN          NaN       NaN   

        tbp_lv_Bext  tbp_lv_C  tbp_lv_Cext  tbp_lv_

array([0.00058486, 0.00043685, 0.00082091, ..., 0.00063237, 0.00061601,
       0.00063082])

### Submission

In [25]:
#df_sub = pd.read_csv("/kaggle/input/isic-2024-challenge/sample_submission.csv")
#df_sub["target"] = lgb_preds
#df_sub.to_csv("submission.csv", index=False)
#df_sub